In [1]:
%config Completer.use_jedi = False

In [2]:
import igraph as ig
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Graph parameters

We're going to build some Erdos-Renyi (ER) models varying its parameters. When we fix a connection probability we're also fixing its degree. So we'll create some realizations for each combination of n and z.

In [3]:
nodes_array = np.arange(1000, 12000, 2000)#[1000, 2000, 4000, 6000, 8000, 10000]
avg_degree_array = [0, 0.5, 0.9, 1, 2, 3, 4, "2log"] #np.arange(0, 3.2, 0.2)
ensemble_size = 30

In [4]:
len(nodes_array) * len(avg_degree_array ) * ensemble_size

5280

In [5]:
def find_percolation_threshold(n = 1000, threshold = 0.5, step = 0.2):
    graphs_dict = dict()
    avg_degree = 0
    while True:
        nodes = n
        graphs_dict[avg_degree] = dict()

        probability = avg_degree/nodes        
        graph = ig.Graph.Erdos_Renyi(n = nodes, p = probability)
        graphs_dict[avg_degree][probability] = graph
        giant = graph.clusters().giant()
        graphs_dict[avg_degree][probability]["giant"] = giant
        giant_fraction = giant.vcount()/n
        avg_degree += step
        if giant_fraction > threshold:
            return graphs_dict

percolations = find_percolation_threshold()

In [6]:
percolations

{0: {0.0: <igraph.Graph at 0x7fea9babf350>},
 0.2: {0.0002: <igraph.Graph at 0x7fea9babf450>},
 0.4: {0.0004: <igraph.Graph at 0x7fea9babf650>},
 0.6000000000000001: {0.0006000000000000001: <igraph.Graph at 0x7fea9babf850>},
 0.8: {0.0008: <igraph.Graph at 0x7fea9babfa50>},
 1.0: {0.001: <igraph.Graph at 0x7fea9babfc50>},
 1.2: {0.0012: <igraph.Graph at 0x7fea9babfe50>},
 1.4: {0.0014: <igraph.Graph at 0x7fea9bb67150>}}

## Graph building

In [7]:
%%time
# A Python program to print all
# permutations using library function
from itertools import product

# Get all permutations of nodes and probabilities
perm = product(*[nodes_array, avg_degree_array])

graphs_dict = dict()
# Print the obtained permutations
for network_property in list(perm):
    
    graphs_list = list()
    nodes = int(network_property[0])
    
    if network_property[1] in ["log", "2log"]:
        degree = 2*np.round(np.log(nodes), 2)
        key = str(nodes) + '_nodes__' + network_property[1] + "_degree"
    else:
        degree = network_property[1]
        key = str(nodes) + '_nodes__' + str(degree) + "_degree"
    probability = degree/nodes
    
    # Create a few realizations
    for i in range(ensemble_size):
        graph = ig.Graph.Erdos_Renyi(n = nodes, p = probability)
        graphs_list.append(graph)
    
    graphs_dict[key] = dict()
    graphs_dict[key]['graphs'] = graphs_list
    graphs_dict[key]["nodes"] = nodes
    graphs_dict[key]["degree"] = network_property[1]
    graphs_dict[key]["probability"] = probability    

CPU times: user 23.8 s, sys: 963 ms, total: 24.7 s
Wall time: 25.1 s


## Calculate statistics and metrics

In [ ]:
%%time
# Calculate some metrics for each graph realization

avg_degrees = list()
nodes_number = list()
connection_probabilities = list()
diameters = list()
sample_realizations = list()
edges_number = list()
giant_size = list()
expected_avg_degree = list()
giant_diameter = list()

for params in graphs_dict.keys():
    i = 1
    for graph in graphs_dict[params]["graphs"]:
        sample_realizations.append(i)
        i += 1
        
        nodes_degree = graph.degree()
        edges_number.append(len(graph.get_edgelist()))
        diameters.append(len(graph.get_diameter()))
        nodes_number.append(graphs_dict[params]["nodes"])
        connection_probabilities.append(graphs_dict[params]["probability"])
        expected_avg_degree.append(graphs_dict[params]["degree"])
        avg_degrees.append(np.mean(nodes_degree))
        giant = graph.clusters().giant()
        giant_size.append(giant.vcount())
        giant_diameter.append(giant.diameter())
        

In [ ]:
# sample_graph = graphs_dict["100_nodes__7_degree"]["graphs"][0]
# sample_graph.vertex_connectivity()

In [ ]:
ensemble_analysis = pd.DataFrame()
ensemble_analysis["nodes_number"] = nodes_number
ensemble_analysis["edges_number"] = edges_number

ensemble_analysis["avg_degree"] = avg_degrees
ensemble_analysis["expected_avg_degree"] = expected_avg_degree
# ensemble_analysis["sample_try"] = sample_realizations

ensemble_analysis["q"] = connection_probabilities
# ensemble_analysis["expected_edges_number"] = (ensemble_analysis["nodes_number"] * (ensemble_analysis["nodes_number"] - 1))*ensemble_analysis["connection_probabilities"]/2

ensemble_analysis["log(n)"] = np.log(ensemble_analysis["nodes_number"])
ensemble_analysis["1/n"] = 1/ensemble_analysis["nodes_number"]
ensemble_analysis["diameter"] = diameters
ensemble_analysis["giant_size"] = giant_size
ensemble_analysis["giant_fraction"] = ensemble_analysis["giant_size"]/ensemble_analysis["nodes_number"]
ensemble_analysis["giant_diameter"] = giant_diameter

ensemble_analysis.head()

In [ ]:
df_plot = ensemble_analysis.pivot_table(index = ["nodes_number", "expected_avg_degree"],
                                         values = ['avg_degree', "edges_number", "diameter", "giant_diameter", "giant_fraction", "q"],
                                         aggfunc = ["mean"])
df_plot

In [ ]:
ensemble_analysis.pivot_table(index = ["expected_avg_degree", "nodes_number"],
                                         values = ['avg_degree', "log(n)", "diameter", "giant_diameter", "giant_fraction", "q"],
                                         aggfunc = ["mean"])

## Can you observe the connectivity threshold?

We saw that if $\frac{q(n)}{q^P(n)} \rightarrow \infty$ P is valid with high probability (whp). Also if we observe that $\frac{q(n)}{q^P(n)} \rightarrow 0$ P is not valid w.h.p., where P is a network property. To have connectivity we should observe

$$\frac{z}{log(n)} \rightarrow \infty$$

So, in order to have connectivity, $z$ should grow at least $log(n)$. If the network is connected, the diameter of the giant component should only increase or remains constant as the number of nodes increases. In the graph we can observe that for the case where the expected average degree is equal to $log(n)$ the giant component diameter is constant. This is not observed for other cases where $\bar{z} = \{0.9, 1.0 \}$. 

In [ ]:
fig, ax = plt.subplots(figsize = (10, 6))

df = df_plot.reset_index()

sns.lineplot(data = df[df["expected_avg_degree"].isin([0.9, 1, "2log"])],
             x = ("nodes_number"),
             y = ("mean", 'diameter'),
             hue = ("expected_avg_degree"),
#              palette = sns.color_palette("vlag", as_cmap=True),
             ax = ax)
ax.set_xlabel('Nodes number')
ax.set_ylabel("Diameter")
ax.set_title("Network diameter at each n")
ax.legend(["z = 0.9", "z = 1", "z = 2log(n)"])
sns.despine()

## Extract the giant components and characterize them

As Erdos and Renyi proved w.h.p, that for large n
- If $q > \frac{1}{n}$ there is one giant component
- If $q < \frac{1}{n}$ there are few small components

Also, we need to define what a giant component is. It should be defined in terms of its size relating to tha entire graph size. 

As defined by 
>*"Loosely speaking, a giant component is a connected component whose relative size (compared to the entire network) does not diminish as the size of the network grows to infinity. As the network grows, the giant component grows with it such that its relative size tends to a non-zero constant."*

In [ ]:
fig, ax = plt.subplots(figsize = (10, 6))

df = df_plot.reset_index()

sns.lineplot(data = df[df["expected_avg_degree"].isin([0.9, 1, "2log"])],
             x = ("nodes_number"),
             y = ("mean", 'giant_diameter'),
             hue = ("expected_avg_degree"),
#              palette = sns.color_palette("vlag", as_cmap=True),
             ax = ax)
ax.set_xlabel('Nodes number')
ax.set_ylabel("Diameter")
ax.set_title("Giant component diameter at each n")
ax.legend(["z = 0.9", "z = 1", "z = 2log(n)"])

sns.despine()

As we can see in the chart above, the size of the giant component does not decrease as the network increase. The chart below displays the proportion of nodes that are in the giant component. As we can see, this proportion does not decrease as the number of nodes increases.

In [ ]:
fig, ax = plt.subplots(figsize = (10, 6))

df = df_plot.reset_index()

sns.lineplot(data = df[df["expected_avg_degree"].isin([0.9, 1, "2log"])],
             x = ("nodes_number"),
             y = ("mean", 'giant_fraction'),
             hue = ("expected_avg_degree"),
#              palette = sns.color_palette("vlag", as_cmap=True),
             ax = ax)
ax.set_xlabel('Nodes number')
ax.set_ylabel("Giant fraction")
ax.set_title("Giant component fraction at each n")
sns.despine()

In [ ]:
def in_giant(G):
    cl = G.components()
    cl_sizes = cl.sizes()
    giant_component_index = cl_sizes.index(max(cl_sizes))
    return [x == giant_component_index for x in cl.membership]

# nodes_in_giant = in_giant(sample_graph)

## Does the network diameter follows the logarithimic variation?

The plot below shows the diameter for each value of n and a fixed value of $z$. As z increases the network diameter fits better the $log(n)$ line. So we can observe that the diameter follows the logarithimic variation.

In [ ]:
# fig, ax = plt.subplots(figsize = (10, 6))

df = df_plot.reset_index()

for z in [0.9, 1, 3, 4]:
    fig, ax = plt.subplots(figsize = (10, 6))
    sns.lineplot(data = df[df["expected_avg_degree"].isin([z])],
                 x = ("nodes_number"),
                 y = ("mean", "diameter"),
    #              hue = "expected_avg_degree",
                 palette = "coolwarm",
                 ax = ax)

    ax2 = ax.twinx()
    plt.plot(df[df["expected_avg_degree"].isin(["2log"])][("nodes_number")],
             np.log(df[df["expected_avg_degree"].isin(["2log"])][("nodes_number")]),
             color = "red", 
             linestyle = "dashed",)


    ax.set_xlabel('Nodes number')
    ax.set_ylabel("Diameter")
#     ax.legend([f"z = {z}"])
    # ax2.legend(["log(n)"], )
    ax2.set_ylabel("log(n)")
    ax.set_title(f"Network diameter variation. z = {z}")
    sns.despine()